TEST EN TS AVEC LANGGRAPH D'UNE ORCHESTRATION DE MULTI AGENTS

In [4]:
import { END, Annotation } from "npm:@langchain/langgraph";
import { BaseMessage } from "npm:@langchain/core/messages";

// This defines the object that is passed between each node
// in the graph. We will create different nodes for each agent and tool
const AgentState = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
        default: () => [],
    }),
    // The agent node that last performed work
    next: Annotation<string>({
        reducer: (x, y) => y ?? x ?? END,
        default: () => END,
    }),
});

In [12]:
process.env.TAVILY_API_KEY = "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";
process.env.MISTRAL_API_KEY = "FivU0jNSyZ0V9oCixR8VRPZ11EMGcUVS";


"FivU0jNSyZ0V9oCixR8VRPZ11EMGcUVS"

In [32]:
// import "npm:tsx"; // Only for running this in TSLab. See: https://github.com/yunabe/tslab/issues/72
import { TavilySearchResults } from "npm:@langchain/community/tools/tavily_search";
import { DynamicStructuredTool } from "npm:@langchain/core/tools";
import * as d3 from "npm:d3";
// ----------ATTENTION----------
// If attempting to run this notebook locally, you must follow these instructions
// to install the necessary system dependencies for the `canvas` package.
// https://www.npmjs.com/package/canvas#compiling
// -----------------------------
// import { createCanvas } from "canvas";
// import { z } from "zod";
// import * as tslab from "tslab";

const additionerTool = new DynamicStructuredTool({
    name: "number_additionner",
    description: "Additionne deux nombres",
    func: (a: number, b: number) => { return a + b},
});

const tavilyTool = new TavilySearchResults();

In [44]:
import { z } from "npm:zod";
import { ChatMistralAI } from "npm:@langchain/mistralai";

import { ChatPromptTemplate, MessagesPlaceholder } from "npm:@langchain/core/prompts";

const members = ["researcher", "number_additionner"] as const;

const systemPrompt =
    "You are a supervisor tasked with managing a conversation between the" +
    " following workers: {members}. Given the following user request," +
    " respond with the worker to act next. Each worker will perform a" +
    " task and respond with their results and status. When finished," +
    " respond with FINISH.";
const options = [END, ...members];

// Define the routing function
const routingTool = {
    name: "route",
    description: "Select the next role.",
    schema: z.object({
        next: z.enum([END, ...members]),
    }),
}

const prompt = ChatPromptTemplate.fromMessages([
    ["system", systemPrompt],
    new MessagesPlaceholder("messages"),
    [
        "human",
        "Given the conversation above, who should act next?" +
        " Or should we FINISH? Select one of: {options}",
    ],
]);

const formattedPrompt = await prompt.partial({
    options: options.join(", "),
    members: members.join(", "),
});

const llm = new ChatMistralAI({
    modelName: "mistral-large-latest",
    temperature: 0,
});

const supervisorChain = formattedPrompt
    .pipe(llm.bindTools(
        [routingTool],
        {
            tool_choice: "any",
        },
    ))
    // select the first one
    .pipe((x) => (x.tool_calls[0].args));

In [43]:
import { HumanMessage } from "npm:@langchain/core/messages";

await supervisorChain.invoke({
    messages: [
        new HumanMessage({
            content: "write a report on birds.",
        }),
    ],
});

{ next: "researcher" }